In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("all_data.csv",sep="\t")

In [ ]:
def prepare_data(x,y):
    x_arr = x.split()
    y_arr = y.split()
    if len(x_arr) != len(y_arr):
        print("problem")
        return (x,{'entities': []})
    
    entities = []
    for a,b in zip(x_arr,y_arr):
        if b != "O":
            entities.append((find_start_end(x,a)[0],find_start_end(x,a)[1],b))
    
    data = (x,{"entities":entities})
    return data


In [ ]:
def find_start_end(string,substring):
    start_idx = string.find(substring)
    end_idx = start_idx + len(substring)
    return start_idx,end_idx

In [ ]:
for line in df["entities"]:
    try:
        print(set(line.split()))
    except:
        pass

In [ ]:
TRAIN_DATA = []
for line,ent in zip(df["clean_tweet"],df["entities"]):
    try:
        TRAIN_DATA.append(prepare_data(line,ent))
    except:
        TRAIN_DATA.append(prepare_data(line,"O "*len(line.split())))

In [ ]:
from tqdm import tqdm
import spacy
import random
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in tqdm(TRAIN_DATA):
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except:
                    pass
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)


In [2]:
test_text = "CVE-2020-1707 linux kernel version 5.4.54 buffer overflow already being exploited in the wild".lower()#input("Enter your testing text: ")
doc = nlp(test_text)
params = {}

for ent in doc.ents:
    lab = ent.label_.split("-")[1]
    if lab not in params:
        params[lab] = ent.text
    else:
        params[lab] = params[lab] + " " + ent.text
        
    print(ent.text, ent.start_char, ent.end_char, ent.label_.split("-")[1])


cve-2020-1707 0 13 ID
linux 14 19 PRO
kernel 20 26 PRO
5.4.54 35 41 VER
buffer 42 48 VUL
overflow 49 57 VUL
